In [2]:
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
import pandas as pd
import numpy as np

db = DatabaseConnector()
table = DataExtractor()


In [3]:
df = table.read_rds_table(db, 'legacy_users')
df = df.drop('index',axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB


In [5]:
df.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB
None


In [7]:
# First process the the date columns 
# Converting any date columns into the date format
import pandas as pd
from dateutil.parser import parse
for col in df.columns:  
    if 'date' in col:
        def try_parsing_date(text):
            try:
                return parse(text)
            except Exception:
                return pd.NaT
        
        df[col] = df[col].apply(try_parsing_date)
        df[col] = pd.to_datetime(df[col], errors='coerce')



In [8]:
from datetime import datetime
# Calculate the current date
current_date = datetime.now()
# Subset the data to show individuals older than 70 years
subset_older_than_70 = df[current_date.year - df['date_of_birth'].dt.year > 80]
print(subset_older_than_70)

      first_name last_name date_of_birth                      company  \
1            Guy     Allen    1940-12-01                      Fox Ltd   
21          Jörg  Hoffmann    1940-11-04                        Höfig   
66        Olivia  Williams    1940-10-23  Sanderson, Singh and Wilson   
77      Klaus-D.      Gute    1942-06-21        Conradi GmbH & Co. KG   
84       Dominic    Potter    1940-01-19                 Powell-Carey   
...          ...       ...           ...                          ...   
15274       Joan      Reed    1942-10-24                    Frost PLC   
15289    Stephen     Davis    1940-08-30    Graham, Fisher and Gibson   
15292    Stephen      Ward    1940-07-22                  Smith-Brown   
15308    Stephen    Hudson    1940-09-13                   Harris PLC   
15317    Stephen  Losekann    1940-10-09                      Rosenow   

                        email_address  \
1        rhodesclifford@henderson.com   
21             zgehringer@beckmann.de   


In [9]:
#Removing NA rows of data
#Further inspection show all NA rows in date_of_birth include non-null rows but are randomised.
nat_indexes = df.index[df['date_of_birth'].isna()].tolist()
print(nat_indexes)
na_list = df.iloc[nat_indexes]
print(na_list)

[752, 866, 1022, 1046, 1805, 2103, 2437, 2739, 2764, 2995, 3536, 4984, 5306, 5307, 6420, 6920, 7737, 8386, 9013, 10013, 10211, 10224, 10360, 10988, 11366, 11443, 11598, 11761, 11864, 12092, 12177, 12584, 13111, 13855, 14101, 14499]
       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84         

In [10]:
# Double checking if the parse and to_datetime function worked on dates that had string e.g. October 
# Anti join show na_list removes the randomised and unparsable dates and leaves the good ones in
result = df[df['date_of_birth'].astype(str).str.contains('[A-Za-z]', regex=True, na=False)]
anti_join_df = result.merge(na_list, left_index=True, right_index=True, how='left', indicator=True)
anti_join_result = anti_join_df[anti_join_df['_merge'] == 'left_only']
print(result)
print(anti_join_result)

       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84           NaT  0YJ2FRMDB4    O5Q6D7FDAF   
3536   E5139W1FJ3  X88YLZX0JX           NaT  648S19YUFF    BYU3R2LQF8   
4984         NULL        NULL           NaT        NULL          NULL   
5306   XKDVQD7BH2  4YSEX8AY1Z           NaT  GKJZ58

In [11]:
# So to conclude, all nulls in date of birth removed, this also removes randomised records of data
df = df.drop(nat_indexes)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   15284 non-null  object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.4+ MB


In [13]:
# This shows no NA values are left. Following scripts would be to look for and handle incorrect data
na_counts = df.isna().sum()
print(na_counts)

first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


In [14]:
# Next fix the phone numbers
# The below processing of US, UK and Germany has been combined into this single cell
us_indexes = df[df['country'] == "United States"].index
other_indexes = df[df['country'] != "United States"].index

df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])', '', regex=True)
df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]', '', regex=True)
df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [16]:
#Review country_code
df['country_code'].value_counts()

country_code
GB     9365
DE     4708
US     1205
GGB       6
Name: count, dtype: int64

In [17]:
df.loc[df['country_code'] == "GGB", 'country_code'] = "GB"

In [18]:
# Next country - all good
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [19]:
# assumption that the address column is correct, could separate postcode and cross-reference on google maps
print(df['address'][0])

Zimmerstr. 1/0
59015 Gießen


In [20]:
# Remove the new line and replace with comma
df['address'] = df['address'].str.replace('\n', ',')
print(df['address'])

0                             Zimmerstr. 1/0,59015 Gießen
1          Studio 22a,Lynne terrace,McCarthymouth,TF0 9GH
2                        92 Ann drive,Joanborough,SK0 6LR
3                    19 Robinson meadow,New Tracy,W22 2QG
4                     3 White pass,Hunterborough,NN96 4UE
                               ...                       
15315         Studio 41I,Jones lodge,Oliviaborough,E8 3DU
15316                 530 Young parkway,Millsfurt,L4G 7NX
15317    Viviane-Fritsch-Straße 3/5,15064 Bad Liebenwerda
15318     660 Ross Falls Suite 357,Anthonymouth, MA 09610
15319         Studio 4,Hancock road,Phillipsview,ST1X 3XB
Name: address, Length: 15284, dtype: object


In [21]:
#All email addresses follow a consistent pattern
df['email_address'].str.contains('@.*\.', regex=True).value_counts()

email_address
True    15284
Name: count, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [23]:

print(df['first_name'].str.contains(r'\d', regex=True).value_counts())
print(df['last_name'].str.contains(r'\d', regex=True).value_counts())
print(df['company'].str.contains(r'\d', regex=True).value_counts())

first_name
False    15284
Name: count, dtype: int64
last_name
False    15284
Name: count, dtype: int64
company
False    15284
Name: count, dtype: int64


In [24]:
df['first_name'].value_counts()

first_name
Michael       57
Amanda        52
Peter         51
Jennifer      50
Andrew        50
              ..
Kristopher     1
Guido          1
Darlene        1
Adem           1
Athanasios     1
Name: count, Length: 2162, dtype: int64

In [25]:
#Below was the analysis of numbers from different countries

In [26]:
# Next fix the phone numbers
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [27]:
#

In [28]:
#10 digits
us = df[df['country'] == 'United States']
print(us)
print(us.info())

      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5              mwilliams@nichols.org   
9                 t

In [29]:
# For the USA, typically follow XXX-XXX-XXXX pattern
us.loc[:,'phone_number'] = us['phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])','',regex=True)
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())
print(us)


valid_phone
False    1205
Name: count, dtype: int64
      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5      

C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\3246566554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10


In [30]:
#Error check - ensure all numbers have exactly 10 digits
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())

valid_phone
False    1205
Name: count, dtype: int64


In [31]:
#Germany have 11 digits
germany = df[df['country'] == "Germany"]
print(germany.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4708 entries, 0 to 15317
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     4708 non-null   object        
 1   last_name      4708 non-null   object        
 2   date_of_birth  4708 non-null   datetime64[ns]
 3   company        4708 non-null   object        
 4   email_address  4708 non-null   object        
 5   address        4708 non-null   object        
 6   country        4708 non-null   object        
 7   country_code   4708 non-null   object        
 8   phone_number   2615 non-null   object        
 9   join_date      4708 non-null   datetime64[ns]
 10  user_uuid      4708 non-null   object        
dtypes: datetime64[ns](2), object(9)
memory usage: 441.4+ KB
None


In [32]:
# Some german numbers are 10 digits. Replace with NaN
germany.loc[:,'phone_number'] = germany['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)

In [33]:
#Error check - ensure all numbers have exactly 10 digits
germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11
germany.loc[~germany['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(germany['valid_phone'].value_counts())


valid_phone
True     2615
False    2093
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\787672041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11


In [34]:
#Lastly the UK
uk = df[df['country'] == "United Kingdom"]
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

In [35]:
uk.loc[:,'phone_number'] = uk['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)
uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11
print(uk['valid_phone'].value_counts())

valid_phone
True     6474
False    2897
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\1538497412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11


In [36]:
uk.loc[~uk['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

In [37]:
#Data cleaning of card details

In [38]:
import tabula

In [39]:
pdf = tabula.read_pdf('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf', pages = 'all')

In [40]:
len(pdf)

279

In [41]:
import pandas as pd
pdf2 = pd.concat(pdf, ignore_index = True)
print(pdf2.head(5))
print(type(pdf2))

        card_number expiry_date                card_provider  \
0    30060773296197       09/26  Diners Club / Carte Blanche   
1   349624180933183       10/23             American Express   
2  3529023891650490       06/23                 JCB 16 digit   
3   213142929492281       09/27                 JCB 15 digit   
4      502067329974       10/25                      Maestro   

  date_payment_confirmed  
0             2015-11-25  
1             2001-06-18  
2             2000-12-26  
3             2011-02-12  
4             1997-03-13  
<class 'pandas.core.frame.DataFrame'>


In [42]:
# Now in dataframe format
print(pdf2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB
None


In [43]:
card_names = pdf2['card_provider'].value_counts()
get_card_names = card_names[card_names > 1000].index.tolist()
print(get_card_names)

['VISA 16 digit', 'JCB 16 digit', 'VISA 13 digit', 'JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']


In [44]:
#All the dodgy cards have dodgy data, so remove
checking_dodgy_cards = pdf2[~pdf2['card_provider'].isin(get_card_names)]
print(checking_dodgy_cards)
print(checking_dodgy_cards.info())

      card_number expiry_date card_provider date_payment_confirmed
377          NULL        NULL          NULL                   NULL
827    VAB9DSB8ZM  NWS3P2W38H    NB71VBAHJE             GTC9KBWJO9
847          NULL        NULL          NULL                   NULL
884          NULL        NULL          NULL                   NULL
1443   MOZOT5Q95V  8YJ3TYH6Z5    WJVMUO4QX6             DJIXF1AFAZ
2418         NULL        NULL          NULL                   NULL
2489         NULL        NULL          NULL                   NULL
2830         NULL        NULL          NULL                   NULL
3694   K0084A9R99  ACT9K6ECRJ    JRPRLPIBZ2             H2PCQP4W50
4196         NULL        NULL          NULL                   NULL
4208   Y8ITI33X30  WDWMN9TU45    TS8A81WFXV             XTD27ANR5Q
4916   RNSCD8OCIM  VNLNMWPJII    JCQMU8FN85             7VGB4DA1WI
5686         NULL        NULL          NULL                   NULL
6024         NULL        NULL          NULL                   

In [45]:
# pdf3 symbolises the removal of nulls and dodgy data
pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]
print(pdf3['card_provider'].value_counts())

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [46]:
print(pdf3['expiry_date'])

0        09/26
1        10/23
2        06/23
3        09/27
4        10/25
         ...  
15304    12/28
15305    11/24
15306    04/24
15307    06/27
15308    02/30
Name: expiry_date, Length: 15284, dtype: object


In [47]:
#pdf3['expiry_date'] = pdf3['expiry_date'].apply(parse)
#pdf3['expiry_date'] = pd.to_datetime(pdf3['expiry_date'], errors='coerce', format = '%m/%Y')
#na_index = pdf3.index[pdf3['date_of_birth'].isna()].tolist() 
pdf3['expiry_date'].head(20)

0     09/26
1     10/23
2     06/23
3     09/27
4     10/25
5     11/23
6     07/27
7     02/29
8     02/24
9     07/23
10    10/28
11    11/27
12    11/31
13    10/29
14    01/29
15    01/26
16    02/32
17    10/26
18    08/28
19    09/32
Name: expiry_date, dtype: object

In [48]:
def custom_parse(date_string):
    month,year = date_string.split('/')
    return parse('01'+'/'+ date_string, ignoretz = True)

In [49]:
pdf3['expiry_date'][1]

'10/23'

In [50]:
pdf3.loc[:,'expiry_date'] = pdf3['expiry_date'].apply(custom_parse)
pdf3.loc[:,'expiry_date'] = pd.to_datetime(pdf3['expiry_date'])

In [51]:
print(pdf3['expiry_date'].head(20))
print(pdf3['expiry_date'].isnull().sum())


0     2026-01-09 00:00:00
1     2023-01-10 00:00:00
2     2023-01-06 00:00:00
3     2027-01-09 00:00:00
4     2025-01-10 00:00:00
5     2023-01-11 00:00:00
6     2027-01-07 00:00:00
7     2029-01-02 00:00:00
8     2024-01-02 00:00:00
9     2023-01-07 00:00:00
10    2028-01-10 00:00:00
11    2027-01-11 00:00:00
12    2031-01-11 00:00:00
13    2029-01-10 00:00:00
14    2029-01-01 00:00:00
15    2026-01-01 00:00:00
16    2032-01-02 00:00:00
17    2026-01-10 00:00:00
18    2028-01-08 00:00:00
19    2032-01-09 00:00:00
Name: expiry_date, dtype: object
0


In [52]:
#No expiry dates our of the ordinary
expired = pdf3[pdf3['expiry_date'] > pd.to_datetime('2050-01-01')]
print(expired)

Empty DataFrame
Columns: [card_number, expiry_date, card_provider, date_payment_confirmed]
Index: []


In [53]:
pdf3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15284 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15284 non-null  object
 3   date_payment_confirmed  15284 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [54]:
# date of payment looks good too
pdf3['date_payment_confirmed'].head(5)
pdf3.loc[:,'date_payment_confirmed'] = pdf3['date_payment_confirmed'].apply(parse)
pdf3['date_payment_confirmed'].isnull().sum()
pdf3[pdf3['date_payment_confirmed'] > pd.to_datetime("2022-12-01")]

,card_number,expiry_date,card_provider,date_payment_confirmed


In [55]:
# Card number
pdf3['card_number'].head(20)
print(type(pdf3['card_number'][1]))

<class 'int'>


In [56]:
def check_alpha(string):
    return str(string).isalpha()

In [57]:
#This didnt pick up some card numbers had '?' 
pdf3['card_number'].apply(check_alpha).value_counts()

card_number
False    15284
Name: count, dtype: int64

In [58]:
pdf3.loc[:,'card_number'] = str(pdf3['card_number'])
print(pdf3['card_provider'].value_counts())
# diners club = 14
# AMEX = 15
# Maestro = 12 to 19. 50, 56-69, 67
# Discover = 16
# Mastercard = 16

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [59]:
def get_length(column):
    if 'Diners' in column:
        return 14
    elif 'American Express' in column:
        return 15
    elif 'Discover' in column:
        return 16
    elif 'Mastercard' in column:
        return 16
    elif 'JCB' in column or 'VISA' in column:
        return int(''.join(filter(str.isdigit, column)))
    elif 'Maestro' in column:
        return list(range(12,20))
    elif 'Diners Club / Carte Blanche' in column:
        return 14
    else:
        return None

In [63]:
pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\3089059518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)


In [61]:
#Run this code if you encounter any errors 
pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]

In [64]:
pdf3[['card_number','card_provider','expected_num_dig']]

,card_number,card_provider,expected_num_dig
0,30060773296197,Diners Club / Carte Blanche,14
1,349624180933183,American Express,15
2,3529023891650490,JCB 16 digit,16
3,213142929492281,JCB 15 digit,15
4,502067329974,Maestro,"[12, 13, 14, 15, 16, 17, 18, 19]"
...,...,...,...
15304,180036921556789,JCB 15 digit,15
15305,180018030448512,JCB 15 digit,15
15306,3569953313547220,JCB 16 digit,16
15307,4444521712606810,VISA 16 digit,16


In [65]:
pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\284436329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)


In [66]:
pdf3['comparison'].value_counts()

comparison
True     15159
False      125
Name: count, dtype: int64

In [ ]:
import pandas as pd
review = pdf3[pdf3['comparison'] == False].head(100)
review.to_csv('review.csv')

In [ ]:
review = pdf3[pdf3['card_provider'] == 'Maestro']
print(review)

        card_number expiry_date card_provider date_payment_confirmed  \
4      502067329974       10/25       Maestro             1997-03-13   
9      502049986008       07/23       Maestro             2011-04-30   
35      60485138139       08/27       Maestro             2022-11-18   
42      60457486839       07/27       Maestro             2006-12-31   
49     676119752563       02/25       Maestro             1995-07-22   
...             ...         ...           ...                    ...   
15240  639064083889       04/32       Maestro             2013-10-03   
15285  675968836147       10/24       Maestro             2009-02-26   
15289  502072745834       11/26       Maestro             2003-08-25   
15290  676167152013       11/28       Maestro             2015-05-22   
15291  675910915411       05/28       Maestro             2002-08-29   

                       expected_num_dig  comparison  
4      [12, 13, 14, 15, 16, 17, 18, 19]        True  
9      [12, 13, 14, 15, 16,

In [67]:
pdf3['card_number'] = pdf3['card_number'].astype(str)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_3656\3786287763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3['card_number'] = pdf3['card_number'].astype(str)


In [68]:
pdf3[pdf3['card_number'].str.contains(r'\D')]


,card_number,expiry_date,card_provider,date_payment_confirmed,expected_num_dig,comparison
149,?4971858637664481,04/24,VISA 16 digit,2016-11-17,16,False
157,???3554954842403828,06/29,JCB 16 digit,1998-01-04,16,False
1777,??4654492346226715,03/23,VISA 16 digit,1996-08-09,16,False
3143,?3544855866042397,10/28,JCB 16 digit,2020-08-25,16,False
4299,??2720312980409662,11/28,Mastercard,1998-04-16,16,False
5289,??4982246481860,01/29,VISA 13 digit,2019-11-30,13,False
5518,?213174667750869,09/26,JCB 15 digit,2000-07-08,15,False
5843,????3505784569448924,02/28,JCB 16 digit,2012-10-12,16,False
6126,????3556268655280464,11/29,JCB 16 digit,1997-01-03,16,False
6382,???2604762576985106,10/29,Mastercard,2015-11-03,16,False


In [ ]:
pdf3.loc[pdf3['card_number'].str.contains(r'\D'), 'card_number'] = None

In [ ]:
print(subset_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 149 to 14346
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             26 non-null     object
 1   expiry_date             26 non-null     object
 2   card_provider           26 non-null     object
 3   date_payment_confirmed  26 non-null     object
 4   expected_num_dig        26 non-null     object
 5   comparison              26 non-null     bool  
dtypes: bool(1), object(5)
memory usage: 1.2+ KB
None


In [ ]:
pdf3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15308
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15284 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15284 non-null  object
 3   date_payment_confirmed  15284 non-null  object
 4   expected_num_dig        15284 non-null  object
 5   comparison              15284 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 731.4+ KB


In [ ]:
subset_df1 = pdf3[pdf3['card_number'].str.contains(r'\n')]

In [ ]:
print(subset_df1)

Empty DataFrame
Columns: [card_number, expiry_date, card_provider, date_payment_confirmed, expected_num_dig, comparison]
Index: []


In [ ]:
pdf3['card_number'].isnull().sum()

0

In [ ]:
zeroone = us[us['phone_number'].str.contains(r'001-',regex=True)]
print(zeroone )

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
us['phone_number'] = us['phone_number'].str.extract(r'(\d{3})[. -]?(\d{3})[. -]?(\d{4})', expand=False).apply(lambda x: ' '.join(filter(lambda y: isinstance(y, str), x)), axis=1)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_12764\3975869603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us['phone_number'] = us['phone_number'].str.extract(r'(\d{3})[. -]?(\d{3})[. -]?(\d{4})', expand=False).apply(lambda x: ' '.join(filter(lambda y: isinstance(y, str), x)), axis=1)


In [ ]:
us_no_number = us[us['phone_number'].str.len()<10]
us_no_number.reset_index(drop = True, inplace = True)
print(us_no_number.index)
print(len(us_no_number))

RangeIndex(start=0, stop=247, step=1)
247


In [ ]:
us.iloc[us_no_number.index]

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
5,David,Torres,1949-08-12,Yang-Stewart,mwilliams@nichols.org,"49226 Edwards Mountains\nNorth Sarah, DE 69608",United States,US,277 664 6389,2002-01-21,53d21f46-1fa4-452f-a023-26aee2aae4d6
9,Emily,Jones,1992-10-09,Serrano-Leblanc,toni24@carlson.com,"012 Andrea Circle Suite 129\nJosephmouth, AS 2...",United States,US,655 421 5915,1998-05-15,caffe463-4918-4f45-a37d-856dc0f15884
16,Sierra,Spencer,1988-11-02,"Watson, Davis and Cooper",justin43@arnold.info,"05559 Vincent Parkway\nSarahstad, ND 91029",United States,US,572 068 8397,2007-03-20,7c8dc654-3994-40b1-a4a2-e55dc7081fe0
27,Bryan,Nielsen,1979-07-29,Drake-Clark,julie41@jennings.com,"0146 Mathis Extension\nWest Marymouth, WY 18605",United States,US,,2001-09-23,e4aae45c-d605-4457-860d-8a460a0a1d69
35,Melissa,Garcia,1946-06-30,"Nelson, Williams and Martinez",alvarezstacy@green-stokes.com,"9288 Patrick Inlet\nJamesfort, CO 48309",United States,US,723 654 4681,2007-01-05,ee0c5f80-2a18-42df-ad88-e898de71dd50
...,...,...,...,...,...,...,...,...,...,...,...
3021,Elizabeth,Patel,1953-01-01,King-Singleton,kmoss@cantrell-best.com,"2465 Ebony View Apt. 181\nCalebport, KY 89888",United States,US,195 454 9244,1994-03-02,7091d21d-a072-4850-b981-db2f1cab144e
3037,Pamela,Grant,1959-06-18,Butler and Sons,daniel55@tucker.net,"4287 Edwards Court Apt. 128\nNew Jennifer, VT ...",United States,US,397 646 1332,2008-09-27,5fe20772-3bc0-4e7b-b790-37c7bbe215f8
3048,Stephanie,Petty,1986-11-15,Butler-Beck,olsenjames@smith.com,"8705 Gray Fort Suite 725\nSherrytown, ID 62541",United States,US,,1997-10-21,2f6aa562-cd35-4f2a-9e46-5dc2326f75b5
3060,Karen,Walters,1995-10-16,"Gilmore, Carrillo and Martin",daniel33@ayala.com,"0908 Roberts Spring Apt. 063\nLake Nicholas, A...",United States,US,481 132 4184,2001-06-24,57e9b695-53b8-431e-a7fc-525117aa45ad


In [ ]:
name_mask = df[df['first_name'].str.len() >11]

In [ ]:
print(name_mask)
print(name_mask.info())

          first_name  last_name date_of_birth                   company  \
53     Frank-Michael     Weimer    1944-05-07       Lorch GmbH & Co. KG   
239     Heinz Dieter     Häring    1999-07-15      Wilmsen Wohlgemut AG   
320     Anne-Kathrin    Rosenow    1973-03-07       Löwer GmbH & Co. KG   
344    Peter-Michael    Heintze    1960-03-01                   Hartung   
366     Klaus-Dieter       Seip    1965-11-09  Mies Stiftung & Co. KGaA   
...              ...        ...           ...                       ...   
15048   Hans-Michael       Rose    1957-01-10            van der Dussen   
15096   Horst-Günter    Zorbach    1980-03-01                   Jungfer   
15161   Klaus-Dieter  Kostolzin    1970-03-05          Söding Bloch GbR   
15187  Peter-Michael     Schaaf    2005-06-07        Wirth Jäntsch GmbH   
15286   Konstantinos     Thanel    1954-08-05       Fritsch Ehlert GmbH   

                          email_address  \
53                   apoelitz@doehn.com   
239          

In [ ]:
result_df = df[df['first_name'].str.contains(r'\d', na=False)]